In [ ]:
import requests
import pprint
import datetime
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
import sqlite3
import slurm2sql
import warnings

warnings.filterwarnings("ignore")

In [ ]:
startdate='2021-03-18'
enddate='2021-03-19'

In [ ]:
 db = sqlite3.connect('test.db')

In [ ]:
# this fails but does get data, just start the notebook again in the next cell
# if you know the error, please fix it
slurm2sql.slurm2sql(db, ['-S', startdate, '-E', enddate, '-a', '-X'])

In [ ]:
df = pd.read_sql('SELECT * FROM slurm', db)

In [ ]:
 df.head(5)

In [ ]:
df.columns

In [ ]:
pd.to_datetime(df["Start"], unit='s', origin='unix')

In [ ]:
df['starttime'] = pd.to_datetime(df.Start,  unit='s', origin='unix')

In [ ]:
df['endtime'] = pd.to_datetime(df.End, unit='s', origin='unix')

In [ ]:
df.columns

In [ ]:
day_idx=pd.date_range(startdate, enddate, freq='H')

In [ ]:
day_idx

In [ ]:
all_jobs_cpu = pd.DataFrame(np.zeros((1,len(day_idx))).T, index=day_idx, columns=['sum'])

In [ ]:
all_jobs_cpu

In [ ]:
all_jobs_cpu = pd.DataFrame(np.zeros((1,len(day_idx))).T, index=day_idx, columns=['sum'])

for enum, jobid in enumerate(df.JobID):
    #print(jobid)
    if (df[df.JobID==jobid].starttime.isnull().values):
        continue
    if (df[df.JobID==jobid].endtime.isnull().values):
        jobendtime=enddate
    else:
        jobendtime=df[df.JobID==jobid].endtime.to_string(index=False)

    hourly_idx=pd.date_range(df[df.JobID==jobid].starttime.to_string(index=False),
                             jobendtime, freq='H')

    jobcpu=pd.DataFrame(df[df.JobID==jobid].ReqCPUS.to_string(index=False),
                        index=hourly_idx, columns=['cpus'])
    
    jobcpu=jobcpu.resample('H').sum()
    jobcpu = pd.to_numeric(jobcpu.cpus, errors='coerce')
    #print(jobcpu)
    #all_jobs_cpu.add(jobcpu)
    all_jobs_cpu[jobid] = jobcpu

In [ ]:
all_jobs_cpu

In [ ]:
cpusperhour = all_jobs_cpu.sum(axis=1)

In [ ]:
cpusperhour

In [ ]:
cpusperhour.plot()

In [ ]:
jobid=4494326

In [ ]:
df[df.JobID==jobid].endtime.isnull().values

In [ ]:
df[df.JobID==7810908].starttime.to_string(index=False)

In [ ]:
hourly_idx=pd.date_range(df[df.JobID==7810908].starttime.to_string(index=False), df[df.JobID==7810908].endtime.to_string(index=False), freq='H')

In [ ]:
hourly_idx

In [ ]:
df[df.JobID==7810908].ReqCPUS.to_string(index=False)

In [ ]:
job=pd.DataFrame(df[df.JobID==7810908].ReqMem.to_string(index=False), index=hourly_idx, columns=['mem'])

In [ ]:
job

In [ ]:
job=job.resample('H').sum()

In [ ]:
job

In [ ]:
#jobmem["7810908"]=job

In [ ]:
# can sum this way
#jobmem[sum]+=job